In [ ]:
!git clone "https://github.com/SriRamGovardhanam/wastedata-Mask_RCNN-multiple-classes.git"

fatal: destination path 'wastedata-Mask_RCNN-multiple-classes' already exists and is not an empty directory.


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install 'h5py==2.10.0' --force-reinstall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
tensorf

In [ ]:
# !pip install -U scikit-image==0.16.2
# !pip install keras==2.1.0
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import shutil, os
def copytree(src = '/content/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN', dst = '/content/', symlinks=False, ignore=None):
    try:
      shutil.rmtree('/content/.ipynb_checkpoints')
    except:
      pass
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
copytree()
# shutil.copytree('/content/drive/MyDrive/batin_data','/content/dataset') # Erencan 
shutil.copytree('/content/drive/MyDrive/Boun_Ders/Cmpe491-492/batin_data','/content/dataset') # Muhammed


'/content/dataset'

In [ ]:
!pip install -U scikit-image==0.16.2
!pip install keras==2.1.0
%tensorflow_version 1.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Keras-2.1.0-py2.py3-none-any.whl (302 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_object_0020.h5")
# 20.epochdan devam etmesi için buradaki pathi güncelle
# COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "logs/object20220512T1412/mask_rcnn_object_0020.h5")

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")


In [ ]:
class CustomConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + toy

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.8

In [ ]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "tumor")
        self.add_class("object", 2, "inner_wall")
        self.add_class("object", 3, "outer_wall")
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        annotations1 = json.load(open(os.path.join(dataset_dir, "all_images_vgg.json")))
        annotations = list(annotations1.values())  # don't need the dict keys

        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            r=a['regions']
            objects=[]
            polygons=[]
            for i in range(len(r)):
               polygons.append(r[str(i)]['shape_attributes'])
               objects.append(r[str(i)]['region_attributes']['label'])
            print("objects:",objects)
            name_dict = {"tumor": 1,"inner_wall": 2,"outer_wall":3} #,"xyz": 3}
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):

        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
import numpy
dataset_train = CustomDataset()
dataset_train.load_custom("/content/dataset", "train")
dataset_train.prepare()
print(len(dataset_train.image_ids))
dictionary = dict()
name = ""
list1=[]
for i in range(len(dataset_train.image_ids)):
  newname = dataset_train.image_info[i]['id'][0:10]
  list2=[]
  image = dataset_train.load_image(i)
  mask, numids = dataset_train.load_mask(i)
  num_ids = numpy.array(numids)
  outer_wall_index = np.where(num_ids==3)
  inner_wall_index = np.where(num_ids==2)
  tumor_index = np.where(num_ids==1)
  if len(outer_wall_index[0]) == 0 or len(inner_wall_index[0]) == 0 or len(tumor_index[0]) == 0:
    continue

  print("outer_wall_index", outer_wall_index)
  print("inner_wall_index", inner_wall_index)
  print("tumor_index", tumor_index)
  inner_count = 0
  outer_count = 0
  outside_count = 0
  tumor = 0
  for k in tumor_index[0]:
    print("k",k)
    for i in range(512):
      for j in range(512):
        if mask[i][j][k] == True:
          tumor += 1
        if (mask[i][j][k] == mask[i][j][inner_wall_index[0][0]]) and (mask[i][j][k] == True):
          inner_count += 1
        if (mask[i][j][k] == mask[i][j][outer_wall_index[0][0]]) and (mask[i][j][k] == True):
          outer_count += 1
        elif (mask[i][j][k] == True):
          outside_count += 1
  list2.append(tumor)
  list2.append(inner_count)
  list2.append(outer_count-inner_count)
  list2.append(outside_count)
  if name == newname :
    list1 = [x + y for (x, y) in zip(list1, list2)]
  else:
    print(newname)
    dictionary[name]=list1
    name=newname
    list1 = list2
  print("inner_count", inner_count)
  print("outer_count", outer_count-inner_count)
  print("outside_count", outside_count)
  if inner_count == outer_count and outside_count == 0:
    print("VIRADS Score: 1 or 2 or 3")
  elif outer_count > inner_count and outside_count == 0:
    print("VIRADS Score: 3 or 4")
  elif outside_count > 0:
    print("VIRADS Score: 5")
  num_ids = np.array(num_ids, dtype=np.int32)

import json
  
with open('/content/convert_all_images_vgg_43-52.txt', 'w') as convert_file:
     convert_file.write(json.dumps(dictionary))


objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2,

KeyboardInterrupt: ignored

In [ ]:
with open('/content/dataset/convert.txt', 'w') as convert_file:
     convert_file.write(json.dumps(dictionary))

In [ ]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("/content/dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("/content/dataset", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40,
                layers='all')

In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [ ]:
weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

... done downloading pretrained model!


In [ ]:
model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
# !pip install keras==2.2.5
# # if error occurs restart runtime
# import keras.backend as K
# K.clear_session()
train(model)
import shutil
shutil.copytree('/content/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs', '/content/drive/MyDrive/MaskRCNN/HorseManLogs')

objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2,

/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2039: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers 

Epoch 1/40
 99/100 [============================>.] - ETA: 2s - loss: 1.4687 - rpn_class_loss: 0.0393 - rpn_bbox_loss: 0.3552 - mrcnn_class_loss: 0.2543 - mrcnn_bbox_loss: 0.4700 - mrcnn_mask_loss: 0.3498

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2197: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'



100/100 [==============================] - 534s 5s/step - loss: 1.4617 - rpn_class_loss: 0.0390 - rpn_bbox_loss: 0.3534 - mrcnn_class_loss: 0.2523 - mrcnn_bbox_loss: 0.4680 - mrcnn_mask_loss: 0.3490 - val_loss: 0.7999 - val_rpn_class_loss: 0.0233 - val_rpn_bbox_loss: 0.2246 - val_mrcnn_class_loss: 0.0697 - val_mrcnn_bbox_loss: 0.2360 - val_mrcnn_mask_loss: 0.2464
Epoch 2/40
100/100 [==============================] - 193s 2s/step - loss: 0.7686 - rpn_class_loss: 0.0237 - rpn_bbox_loss: 0.2136 - mrcnn_class_loss: 0.0699 - mrcnn_bbox_loss: 0.2276 - mrcnn_mask_loss: 0.2339 - val_loss: 0.7228 - val_rpn_class_loss: 0.0159 - val_rpn_bbox_loss: 0.2003 - val_mrcnn_class_loss: 0.0727 - val_mrcnn_bbox_loss: 0.2026 - val_mrcnn_mask_loss: 0.2312
Epoch 3/40
100/100 [==============================] - 193s 2s/step - loss: 0.7142 - rpn_class_loss: 0.0187 - rpn_bbox_loss: 0.2359 - mrcnn_class_loss: 0.0679 - mrcnn_bbox_loss: 0.1805 - mrcnn_mask_loss: 0.2112 - val_loss: 0.5836 - val_rpn_class_loss: 0.011

In [ ]:
import shutil
shutil.copytree('/content/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs', '/content/drive/MyDrive/MaskRCNN/HorseManLogs')

'/content/drive/MyDrive/MaskRCNN/HorseManLogs'

# **TESTING**


In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg


# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

WEIGHTS_PATH = "/content/drive/MyDrive/MaskRCNN/Train_40_2-11/mask_rcnn_object_0026.h5"  # TODO: update this path

In [ ]:
config = CustomConfig()
CUSTOM_DIR = os.path.join(ROOT_DIR, "/content/dataset/")
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.8

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.8
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [ ]:

DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
CUSTOM_DIR = "/content/dataset"
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR, "val")

dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['inner_wall', 'outer_wall']
numids [2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2, 3]
objects: ['tumor', 'inner_wall', 'outer_wall']
numids [1, 2,

In [ ]:
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)



Instructions for updating:
Use `tf.cast` instead.


In [ ]:
weights_path = WEIGHTS_PATH

print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  /content/drive/MyDrive/MaskRCNN/Train_40_2-11/mask_rcnn_object_0026.h5








In [ ]:
# #RUN DETECTION
# image_id = random.choice(dataset.image_ids)
# print(image_id)
# image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#     modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
# info = dataset.image_info[image_id]
# print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
#                                        dataset.image_reference(image_id)))

# results = model.detect([image], verbose=1)

# ax = get_ax(1)
# r = results[0]
# visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset.class_names, r['scores'], ax=ax,
#                             title="Predictions")
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

import os
rootdir = '/content/dataset/test/batin'
dictionary = dict()
name = ""
list1=[]
# for subdir, dirs, files in os.walk(rootdir):
for i in range(2,23):
  director = rootdir + str(i)
  for filename in os.listdir(director):
    f = os.path.join(director, filename)
    if (f.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))):
      image1 = mpimg.imread(f)
      print(filename)
    # for i in range(23,24):
      # image1 = mpimg.imread('/content/dataset/test/batin46/KAS18384298_ep2d_diff_tra_0-800-1000-_CALC_BVAL_DFC_MIX_' + str(i) + '.jpg')

          # Run object detection
      # print("length",len(image1))
      # print("shape",image1.shape)
      # image1 = cv2. cvtColor(image1,cv2. COLOR_GRAY2RGB)
      results1 = model.detect([image1], verbose=1)

          # Display results
      ax = get_ax(1)
      r1 = results1[0]
      # outer_wall_index = np.where(r1['class_ids'] == 3);
      # inner_wall_index = np.where(r1['class_ids'] == 2);
      # tumor_index = np.where(r1['class_ids'] == 1);

      # inner_count = 0
      # outer_count = 0
      # outside_count = 0
      # for i in range(512):
      #   for j in range(512):
      #     if (r1['masks'][i][j][tumor_index[0]] == r1['masks'][i][j][inner_wall_index[0]]) and (r1['masks'][i][j][tumor_index[0]] == True):
      #       inner_count += 1
      #     if (r1['masks'][i][j][tumor_index[0]] == r1['masks'][i][j][outer_wall_index[0]]) and (r1['masks'][i][j][tumor_index[0]] == True):
      #       outer_count += 1
      #     elif (r1['masks'][i][j][tumor_index[0]] == True):
      #       outside_count += 1
      # print("inner_count", inner_count)
      # print("outer_count", outer_count)
      # print("outside_count", outside_count)
      # if inner_count == outer_count and outside_count == 0:
      #   print("VIRADS Score: 1")
      # if inner_count == 0 and outer_count > 0 and outside_count == 0:
      #   print("VIRADS Score: 3")
      # elif inner_count > 0 and outer_count > 0 and outside_count == 0:
      #   print("VIRADS Score: 4")
      # elif outside_count > 0:
      #   print("VIRADS Score: 5")


     
      newname = filename[0:10]
      list2=[]
      num_ids = r1['class_ids']
      mask = r1['masks']

      num_ids = numpy.array(num_ids)
      outer_wall_index = np.where(num_ids==3)
      inner_wall_index = np.where(num_ids==2)
      tumor_index = np.where(num_ids==1)
      if len(outer_wall_index[0]) == 0 or len(inner_wall_index[0]) == 0 or len(tumor_index[0]) == 0:
        continue

      # print("outer_wall_index", outer_wall_index)
      # print("inner_wall_index", inner_wall_index)
      # print("tumor_index", tumor_index)
      inner_count = 0
      outer_count = 0
      outside_count = 0
      tumor = 0
      for k in tumor_index[0]:
        print("k",k)
        for i in range(512):
          for j in range(512):
            if mask[i][j][k] == True:
              tumor += 1
            if (mask[i][j][k] == mask[i][j][inner_wall_index[0][0]]) and (mask[i][j][k] == True):
              inner_count += 1
            if (mask[i][j][k] == mask[i][j][outer_wall_index[0][0]]) and (mask[i][j][k] == True):
              outer_count += 1
            elif (mask[i][j][k] == True):
              outside_count += 1
      list2.append(tumor)
      list2.append(inner_count)
      list2.append(outer_count-inner_count)
      list2.append(outside_count)
      if newname in dictionary:
        dictionary[newname].append(list2)
      else:
        dictionary[newname] = [list2]
      # if name == newname :
      #   # list1 = [x + y for (x, y) in zip(list1, list2)]
      #   # list1.append([0,0,0,0])
      #   # for asd in range(4):
      #   #   list1[-1][asd] += list2[asd]
      #   list1.append(list2)
      # else:
      #   print(newname)
      #   dictionary[name]=list1
      #   name=newname
      #   list1 = list2
      # print("inner_count", inner_count)
      # print("outer_count", outer_count-inner_count)
      # print("outside_count", outside_count)
      # if inner_count == outer_count and outside_count == 0:
      #   print("VIRADS Score: 1 or 2 or 3")
      # elif outer_count > inner_count and outside_count == 0:
      #   print("VIRADS Score: 3 or 4")
      # elif outside_count > 0:
      #   print("VIRADS Score: 5")
      num_ids = np.array(num_ids, dtype=np.int32)
      print(dictionary)
      
      visualize.display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'],
                                  dataset.class_names, r1['scores'], ax=ax,
                                  title="Predictions")

import json
  
with open('/content/convert.txt', 'w') as convert_file:
    convert_file.write(json.dumps(dictionary))

      